In [18]:
import multiprocessing as mp
mp.set_start_method('fork', force=True)

In [ ]:
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: mps


In [6]:
import os
import pandas as pd

In [ ]:
# paths to CSV files for groundtruths and metadata
train_gt = pd.read_csv("/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Training_Part3_GroundTruth.csv")
val_gt = pd.read_csv("/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Validation_Part3_GroundTruth.csv")

train_meta = pd.read_csv("/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Training_Data/ISIC-2017_Training_Data_metadata.csv")
val_meta = pd.read_csv("/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data_metadata.csv")

# mergeing the labels and metadata on image_id
train_df = pd.merge(train_gt, train_meta, on="image_id", how="left")
val_df = pd.merge(val_gt, val_meta, on="image_id", how="left")

# attaching image paths
base_train = "/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Training_Data"
base_val = "/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Validation_Data"

train_df["image_path"] = train_df["image_id"].apply(lambda x: os.path.join(base_train, f"{x}.jpg"))
val_df  ["image_path"] = val_df  ["image_id"].apply(lambda x: os.path.join(base_val,   f"{x}.jpg"))

# checking the above operations
print(train_df.head())
print([os.path.exists(p) for p in train_df["image_path"].head(5)])

       image_id  melanoma  seborrheic_keratosis age_approximate     sex  \
0  ISIC_0000000       0.0                   0.0              55  female   
1  ISIC_0000001       0.0                   0.0              30  female   
2  ISIC_0000002       1.0                   0.0              60  female   
3  ISIC_0000003       0.0                   0.0              30    male   
4  ISIC_0000004       1.0                   0.0              80    male   

                                          image_path  
0  /Users/riyazshaik/Documents/nanu_challenge/ISI...  
1  /Users/riyazshaik/Documents/nanu_challenge/ISI...  
2  /Users/riyazshaik/Documents/nanu_challenge/ISI...  
3  /Users/riyazshaik/Documents/nanu_challenge/ISI...  
4  /Users/riyazshaik/Documents/nanu_challenge/ISI...  
[True, True, True, True, True]


In [47]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

In [60]:
from tqdm.auto import tqdm

In [52]:
# metadata preprocessing
for df in (train_df, val_df):
    df["age_approximate"] = pd.to_numeric(df["age_approximate"], errors="coerce")
    
age_imp = SimpleImputer(strategy="median")
train_df["age_scaled"] = age_imp.fit_transform(train_df[["age_approximate"]])
val_df["age_scaled"] = age_imp.transform(val_df[["age_approximate"]])

age_scl = StandardScaler()
train_df["age_scaled"] = age_scl.fit_transform(train_df[["age_scaled"]])
val_df["age_scaled"] = age_scl.transform(val_df[["age_scaled"]])

for col in train_df.columns:
    if col.startswith("sex_") and col not in val_df.columns:
        val_df[col] = 0
for col in val_df.columns:
    if col.startswith("sex_") and col not in train_df.columns:
        train_df[col] = 0

meta_cols = ["age_scaled"] + [c for c in train_df.columns if c.startswith("sex_")]

for df in (train_df, val_df):
    df[meta_cols] = (
        df[meta_cols]
        .apply(pd.to_numeric, errors="coerce")
        .fillna(0)
        .astype(np.float32)
    )

# useful transforms
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ]),
}

In [53]:
# custom dataset
class ISICMetaDataset(Dataset):
    def __init__(self, df, meta_cols, transform=None):
        self.df = df.reset_index(drop=True)
        self.meta_cols = meta_cols
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # load & transform images
        img = Image.open(row["image_path"]).convert("RGB")
        img = self.transform(img) if self.transform else img

        # metadata extraction
        import numpy as np
        meta_vals = np.array(row[self.meta_cols].values, dtype=np.float32)
        meta = torch.from_numpy(meta_vals)

        label = torch.tensor(int(row["melanoma"]), dtype=torch.long)
        return img, meta, label

# dataloaders for training and validation
batch_size = 16
train_ds = ISICMetaDataset(train_df, meta_cols, transform=data_transforms["train"])
val_ds = ISICMetaDataset(val_df, meta_cols, transform=data_transforms["val"])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)

In [54]:
# checking the operations are working (learned to always check - hard way through experience)
imgs, metas, labels = next(iter(train_loader))
print("imgs:", imgs.shape, "metas:", metas.shape, metas.dtype, "labels:", labels.shape)

imgs: torch.Size([16, 3, 224, 224]) metas: torch.Size([16, 4]) torch.float32 labels: torch.Size([16])


In [55]:
# efficient channel attention implementation
class ECA(nn.Module):
    def __init__(self, channels, k_size=3):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = y.squeeze(-1).transpose(1, 2)
        y = self.conv(y)
        y = self.sigmoid(y).transpose(1, 2)
        y = y.unsqueeze(-1)
        return x * y.expand_as(x)

# feature wise linear modulation implementation
class FiLM(nn.Module):
    def __init__(self, feat_ch, meta_dim):
        super().__init__()
        self.gamma = nn.Linear(meta_dim, feat_ch)
        self.beta = nn.Linear(meta_dim, feat_ch)

    def forward(self, x, meta):
        g = self.gamma(meta).unsqueeze(-1).unsqueeze(-1)
        b = self.beta(meta).unsqueeze(-1).unsqueeze(-1)
        return x * (1 + g) + b

# hybrid model: efficientnet-b3 + ECA + FiLM + Meta-MLP + classifier
class EnhancedHybridECAFiLM(nn.Module):
    def __init__(self, meta_feat_dim, num_classes=2, backbone_name="efficientnet_b3"):
        super().__init__()
        # efficientnet backbone
        self.backbone = timm.create_model(backbone_name, pretrained=True, features_only=True)
        channels = self.backbone.feature_info.channels()
        fin_ch = channels[-1]

        # ECA on final feature map
        self.eca = ECA(fin_ch, k_size=3)
        self.film = FiLM(fin_ch, meta_feat_dim)
        self.pool = nn.AdaptiveAvgPool2d(1)

        # metadata mlp
        self.meta_mlp = nn.Sequential(
            nn.Linear(meta_feat_dim, fin_ch),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
        )

        # classification head
        self.classifier = nn.Sequential(
            nn.Linear(fin_ch, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self, img, meta):
        feats = self.backbone(img)
        fin_feat = feats[-1]
        x = self.eca(fin_feat)
        x = self.film(x, meta)
        x = self.pool(x).flatten(1)
        m = self.meta_mlp(meta)

        out = self.classifier(x + m)
        return out

meta_feat_dim = len(meta_cols)
model = EnhancedHybridECAFiLM(meta_feat_dim=meta_feat_dim, num_classes=2).to(device)
print(model)

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


EnhancedHybridECAFiLM(
  (backbone): EfficientNetFeatures(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNormAct2d(
            40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
  

In [64]:
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve

In [65]:
# training setup using focal loss and one cycle lr scheduler
# focal loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, logits, targets):
        ce = nn.functional.cross_entropy(logits, targets, reduction='none')
        p = torch.exp(-ce)
        loss = self.alpha * (1 - p)**self.gamma * ce
        return loss.mean() if self.reduction=='mean' else loss.sum()

criterion = FocalLoss(gamma=2.0, alpha=0.25)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)

# one cycle lr scheduler
epochs = 20
steps_per_ep = len(train_loader)
scheduler = OneCycleLR(optimizer, max_lr=3e-4, total_steps=epochs * steps_per_ep, pct_start=0.3, anneal_strategy='cos',
                          div_factor=10,
                          final_div_factor=100)

def train_epoch(model, loader, criterion, optimizer, scheduler, device, epoch, total_epochs):
    model.train()
    running_loss = 0.0
    loop = tqdm(loader, desc=f"Epoch {epoch}/{total_epochs} [train]", leave=False)
    for imgs, metas, labels in loop:
        imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
        logits = model(imgs, metas)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() * imgs.size(0)
        loop.set_postfix({
            "loss": f"{loss.item():.4f}",
            "lr": f"{optimizer.param_groups[0]['lr']:.2e}"
        })
    return running_loss / len(loader.dataset)

def validate(model, loader, device, min_spec=0.50):
    model.eval()
    y_true, y_prob = [], []
    loop = tqdm(loader, desc=" [val] ", leave=False)
    with torch.no_grad():
        for imgs, metas, labels in loop:
            imgs, metas = imgs.to(device), metas.to(device)
            logits = model(imgs, metas)
            probs = F.softmax(logits, dim=1)[:, 1]
            y_true.extend(labels.tolist())
            y_prob.extend(probs.cpu().tolist())

            if len(labels.unique()) > 1:
                loop.set_postfix(
                    {"batch AUC": f"{roc_auc_score(labels.cpu(), probs.cpu()):.3f}"}
                )
    # computing metrics
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    auc = roc_auc_score(y_true, y_prob)
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    spec = 1 - fpr
    mask = spec >= min_spec
    best = np.argmax(tpr * mask) if mask.any() else np.argmax(tpr)
    thr_used = thr[best]
    sensitivity = tpr[best]
    specificity = spec[best]

    return auc, sensitivity, specificity, thr_used

best_auc = 0.0
for epoch in range(1, epochs + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, device, epoch, epochs)
    val_auc, val_sens, val_spec, thr_used = validate(model, val_loader, device, min_spec=0.50)

    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), "best_model.pth")

    print(
        f"[epoch {epoch:02d}] "
        f"train loss: {train_loss:.4f} | "
        f"val AUC: {val_auc:.4f} | "
        f"sens: {val_sens:.4f} | "
        f"spec: {val_spec:.4f} | "
        f"thr: {thr_used:.3f}"
    )

Epoch 1/20 [train]:   0%|          | 0/125 [00:00<?, ?it/s]

 [val] :   0%|          | 0/10 [00:00<?, ?it/s]

[epoch 01] train loss: 0.0206 | val AUC: 0.8397 | sens: 0.9000 | spec: 0.5333 | thr: 0.195


Epoch 2/20 [train]:   0%|          | 0/125 [00:00<?, ?it/s]

 [val] :   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 18/20 [train]:   0%|          | 0/125 [00:00<?, ?it/s]

 [val] :   0%|          | 0/10 [00:00<?, ?it/s]

[epoch 18] train loss: 0.0010 | val AUC: 0.8747 | sens: 1.0000 | spec: 0.5000 | thr: 0.014


Epoch 19/20 [train]:   0%|          | 0/125 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [69]:
# performing operations similar to train and validate on test dataset 
test_gt = pd.read_csv("/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Test_v2_Part3_GroundTruth.csv")
test_meta = pd.read_csv("/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_metadata.csv")

# mergeing the labels and metadata on image_id
test_df = pd.merge(test_gt, test_meta, on="image_id", how="left")
base_test = "/Users/riyazshaik/Documents/nanu_challenge/ISIC-2017_Test_v2_Data"
test_df["image_path"] = test_df["image_id"].apply(lambda x: os.path.join(base_test, f"{x}.jpg"))

# metadata preprocessing
test_df["age_approximate"] = pd.to_numeric(test_df["age_approximate"], errors="coerce")
test_df["age_scaled"] = age_imp.transform(test_df[["age_approximate"]])
test_df["age_scaled"] = age_scl.transform(test_df[["age_scaled"]])
test_df = pd.get_dummies(test_df, columns=["sex"], prefix="sex", dummy_na=False)
for col in [c for c in meta_cols if c.startswith("sex_")]:
    if col not in test_df.columns:
        test_df[col] = 0

test_df[meta_cols] = test_df[meta_cols].astype(np.float32)

# dataloader for test dataset
test_ds = ISICMetaDataset(test_df, meta_cols, transform=data_transforms["val"])
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False, num_workers=0)

In [73]:
from sklearn.metrics import accuracy_score, classification_report

In [74]:
# loading the best model for testing
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.to(device).eval()

image_ids, probs = [], []
with torch.no_grad():
    for imgs, metas, _ in test_loader:
        imgs, metas = imgs.to(device), metas.to(device)
        logits = model(imgs, metas)
        batch_probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        probs.extend(batch_probs)
        start = len(image_ids)
        image_ids.extend(test_df["image_id"].iloc[start:start + len(batch_probs)].tolist())

# building results dataframe and merging with ground truth for evaluation
results_df = pd.DataFrame({
    "image_id": image_ids,
    "prob_melanoma": probs
}).merge(test_df[["image_id", "melanoma"]], on="image_id")

# computing predicted label at 0.5
results_df["pred_label"] = (results_df["prob_melanoma"] > 0.5).astype(int)
results_df["correct"] = results_df["pred_label"] == results_df["melanoma"]

acc = accuracy_score(results_df["melanoma"], results_df["pred_label"])
tn, fp, fn, tp = confusion_matrix(results_df["melanoma"], results_df["pred_label"]).ravel()
sens = tp / (tp + fn)
spec = tn / (tn + fp)

print(f"test Accuracy @0.5 : {acc:.3f}")
print(f"test Sensitivity @0.5 : {sens:.3f}")
print(f"test Specificity @0.5 : {spec:.3f}")
print("\nClassification Report:\n", classification_report(results_df["melanoma"], results_df["pred_label"]))
results_df.to_csv("test_results_with_truth.csv", index=False)
print("saved test_results_with_truth.csv with", len(results_df), "rows")

test Accuracy @0.5 : 0.848
test Sensitivity @0.5 : 0.632
test Specificity @0.5 : 0.901

Classification Report:
               precision    recall  f1-score   support

         0.0       0.91      0.90      0.91       483
         1.0       0.61      0.63      0.62       117

    accuracy                           0.85       600
   macro avg       0.76      0.77      0.76       600
weighted avg       0.85      0.85      0.85       600

saved test_results_with_truth.csv with 600 rows
